In [8]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from zipfile import ZipFile
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tfds

In [3]:
#JUPYTER VERSION
#comb_path = "../Data/Combined_Images"

#GOOGLE COLAB VERSION
from google.colab import drive
drive.mount('/content/drive')

# Unzipping the dataset file facial-age.zip

combined_images_path = "/content/drive/MyDrive/Data/Combined_Images.zip"

with ZipFile(combined_images_path, 'r') as myzip:
    myzip.extractall("../content/Combined_Images")
    print('Done unzipping Combined_Images.zip')

comb_path = '../content/Combined_Images' 
batch_size = 64

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Done unzipping Combined_Images.zip


In [5]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  comb_path,
  validation_split=0.2,
  subset="training", #If should be return the training set (80%) or the validation set (20%)
  seed=41, #Seed should guarantee that train_ds and val_ds doesn't have common images
  shuffle=True,
  image_size=(200, 200),
  batch_size=batch_size,
  color_mode='grayscale')

val_ds = tf.keras.utils.image_dataset_from_directory(
  comb_path,
  validation_split=0.2,
  subset="validation",
  seed=41, 
  shuffle=True,
  image_size=(200, 200),
  batch_size=batch_size,
  color_mode='grayscale')

test_dataset = val_ds.take(53)
val_ds = val_ds.skip(53)

print('Batches for training -->', train_ds.cardinality())
print('Batches for validating -->', val_ds.cardinality())
print('Batches for testing -->', test_dataset.cardinality())   

Found 33884 files belonging to 8 classes.
Using 27108 files for training.
Found 33884 files belonging to 8 classes.
Using 6776 files for validation.
Batches for training --> tf.Tensor(424, shape=(), dtype=int64)
Batches for validating --> tf.Tensor(53, shape=(), dtype=int64)
Batches for testing --> tf.Tensor(53, shape=(), dtype=int64)


In [4]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        rotation_range=25
        )
test_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
def convert_tf_to_pd(ds, limit=32):
    """
    Read data from Tensorflow dataset to Pandas dataframe
    :param ds:
    :param limit:
    :return:
    """
    batch_iterator = ds.batch(limit).make_one_shot_iterator()
    with tf.Session() as sess:
        batch = batch_iterator.get_next()
        features_and_labels = sess.run(batch)
        samples = {
            **features_and_labels[0],
            'label': features_and_labels[1],
        }

    return pd.DataFrame.from_dict(samples)

In [6]:
train_generator = train_datagen.flow_from_dataframe(
        train_ds,
        target_size=(200, 200),
        batch_size=batch_size,
        class_mode='categorical'
        )
validation_generator = test_datagen.flow_from_dataframe(
        val_ds,
        target_size=(200, 200),
        batch_size=batch_size,
        class_mode='categorical'
        )

AttributeError: ignored